![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=TechnologyStudies/ComputingScience/Courses/procedural-programming-1.ipynb&depth=1"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# CSE2110: Procedural Programming 1

Students develop their understanding of the procedural programming paradigm. They move from a structured programming approach in which modules were handled through the use of program blocks to a more formal modular programming approach in which they are handled through subprograms. In the process, students also learn to use a number of new design approaches made possible by the new paradigms. As part of this process, they also learn what types of problems are amenable to modular algorithms and programs.

*Prerequisite: [CSE1120: Structured Programming 2](structured-programming-2.ipynb)*



Outcomes: The student will:

1. demonstrate an understanding of modular programming
    1. describe the advantages of programming with modules or subroutines including:
        1. reducing the duplication of code in a program
        2. enabling the reuse of code in more than one program
        3. decomposing complex problems into simpler pieces to improve maintainability and extendibility
        4. improving the readability of a program
        5. hiding or protecting the program data
    2. select a programming environment and describe how it supports procedural programming including:
        1. the type of subprograms supported; e.g., procedures, functions, methods
        2. the level or type of modularity provided
        3. the level of protection provided from unwanted side-effects
<br><br>
2. demonstrate basic procedural programming skills by writing algorithms employing a modular approach to solve problems
    1. analyze a data processing problem and use a top-down design approach to decompose it into discreet input, processing and output modules
    2. analyze and refine modules into submodules that are a manageable size for each process; e.g., input submodules, processing submodules and output submodules
    3. describe and represent, using pseudocode or an appropriate diagramming approach, the relationship among the modules
    4. analyze and rewrite algorithms for each module identifying the pre- and post-conditions and required program control of flow mechanisms.
    5. analyze and evaluate algorithms for each developing module with appropriate data and revise, as required
<br><br>
3. translate algorithms into source code, convert the source code into machine executable form, execute and debug, as required
    1. convert an algorithm into a program of linked subprograms with a main or client module calling other modules in a manner that reflects the structure of the algorithm
    2. use appropriate types of subprograms to implement the various sections of the algorithm; e.g., functions (subprograms that return a value) and procedures (subprograms that do not return a value)
    3. analyze and determine the type of scope required to protect and/or hide data and keep implementation decoupled from the calling modules and to avoid unwanted side-effects with consideration to:
        1. use of appropriate parameters for importing and exporting data to and from subprograms
        2. use of local variables and nested subprograms to enhance cohesion
        3. one- and two-way parameter passing for importing and exporting data to and from subprograms
    4. analyze for, and maintain, an appropriate balance between the coupling or dependency and cohesion or focus of subprograms
    5. create both internal and external documentation
    6. analyze the program and eliminate syntax, run-time and logic errors by using appropriate test data for each programming task at each stage of development
<br><br>
4. compare the results of the program with the intent of the algorithm and modify, as required
    1. use appropriate error trapping mechanisms built into the programming environment, as well as programmer-directed error-trapping techniques, to eliminate logic errors and debug the program
    2. compare the congruency between the outcomes of the debugged program and the original intent of the algorithm and modify, as required
<br><br>
5. demonstrate basic competencies
    1. demonstrate fundamental skills to:
        1. communicate
        2. manage information
        3. use numbers
        4. think and solve problems
    2. demonstrate personal management skills to:
        1. demonstrate positive attitudes and behaviours
        2. be responsible
        3. be adaptable
        4. learn continuously
        5. work safely
    3. demonstrate teamwork skills to:
        1. work with others
        2. participate in projects and tasks
<br><br>
6. identify possible life roles related to the skills and content of this cluster
    1. recognize and then analyze the opportunities and barriers in the immediate environment
    2. identify potential resources to minimize barriers and maximize opportunities

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)